# Table
Construction of m*n game board saved in tab numpy aray .

In [1]:
import numpy as np
import pandas as pd
# hardness='easy'
hardness='veryeasy'
# hardness='medium'
# hardness='hard'
# hardness='expert'
df=pd.read_csv(hardness+'/tab.csv')
tab=np.array(df.values.tolist())
df=pd.read_csv(hardness+'/constrain.csv')
constrain=df.values.tolist()

Table display

In [2]:
tab

array([[-1, -1, -1, -1, -1],
       [-1, -1,  0,  0,  0],
       [-1,  0,  0,  0,  0],
       [-1,  0,  0, -1,  0],
       [-1, -1,  0,  0, -1]])

constrain saved in a list named as constrain .

In [3]:
constrain

[[1, 1, 3, 0, 12],
 [3, 0, 2, 0, 3],
 [4, 1, 2, 0, 17],
 [4, 4, 2, 0, 10],
 [0, 2, 4, 1, 20],
 [0, 3, 2, 1, 3],
 [0, 4, 3, 1, 23],
 [1, 1, 2, 1, 5],
 [3, 3, 1, 1, 8],
 [3, 3, 1, 0, 6],
 [2, 0, 4, 0, 21]]

checking if table hoped to be solved ?

In [4]:
def constrain_hope(table,constrain):
    if constrain==True:
        return True
    (i,j)=(constrain[0],constrain[1])
    n=constrain[2]
    sum=constrain[4]
    if constrain[3]:
        r=table[i+1:i+1+n,j]      
    else:
        r=table[i,j+1:j+1+n]
    if (np.unique(r[r!=0]).size)<(r[r!=0].size):
        return False
    if sum<np.sum(r):
        return False
    return True

checking if is completely solved ?

In [5]:
def tab_solved(table,constrain):
   for c in constrain:
      (i,j)=(c[0],c[1])
      n=c[2]
      sum=c[4]
      if c[3]:
         r=table[i+1:i+1+n,j]      
      else:
         r=table[i,j+1:j+1+n]
      if not(sum==np.sum(r)):
         return False
      return True

searching a cell constrain in raow and column 

In [6]:
def search_constrain_v(constrain,x,y):
    for i in constrain:
        if i[3]==True and i[1]==y and x>i[0] and x<=i[0]+i[2]:
            return i
    return True
def search_constrain_h(constrain,x,y):
    for i in constrain:
        if i[3]==False and i[0]==x and y>i[1] and y<=i[1]+i[2]:
            return i
    return True

# search
The below function is to search in state space .

In [7]:
def search(tab,constrain,selection):
    r,c=selection(tab)
    if r==-1:
        if tab_solved(tab,constrain):
            return True
        else :
            return False
    c1=search_constrain_v(constrain,r,c)
    c2=search_constrain_h(constrain,r,c)
    for i in range(1,9):
        tab[r,c]=i
        if not(constrain_hope(tab,c1) and constrain_hope(tab,c2)):
            tab[r,c]=0
            continue
        if search(tab,constrain,selection):
            return True
        tab[r,c]=0
    return False

# primary agent
selection in order :

In [8]:
def select_cell(tab):
    if not 0 in tab:
        return -1,-1
    else:
        for i in range(tab.shape[0]):
            for j in range(tab.shape[1]):
                if tab[i,j]==0:
                    return i,j

In [9]:
tab1=tab.copy()
search(tab1,constrain,selection=select_cell)

True

In [10]:
tab1

array([[-1, -1, -1, -1, -1],
       [-1, -1,  3,  1,  8],
       [-1,  1,  4,  2,  3],
       [-1,  2,  1, -1,  1],
       [-1, -1,  2,  1, -1]])

selction in random

In [11]:
def select_cell_random(tab):
    if not 0 in tab:
        return -1,-1
    else:
        while True:
            a=np.random.randint(0,tab.shape[0],size=1)
            b=np.random.randint(0,tab.shape[1],size=1)
            if tab[a,b]==0:
                return a,b

In [18]:
tab2=tab.copy()
search(tab2,constrain,selection=select_cell_random)

True

In [19]:
tab2

array([[-1, -1, -1, -1, -1],
       [-1, -1,  3,  2,  7],
       [-1,  3,  4,  1,  2],
       [-1,  1,  2, -1,  1],
       [-1, -1,  1,  2, -1]])

# Intelligent agent

In [14]:
def select_cell_intelligent(tab,constrain=constrain):
    if not 0 in tab:
        return -1,-1
    m=select_cell(tab)
    c_m=[]
    cv_m=search_constrain_v(constrain,m[0],m[1])
    ch_m=search_constrain_h(constrain,m[0],m[1])
    if type(cv_m)==bool and type(ch_m)==bool:
        print('dfsdf',m)
    if type(cv_m)==bool or (not type(ch_m)==bool and ch_m[4]<cv_m[4]):
        c_m=ch_m.copy()
    if type(ch_m)==bool or(not type(cv_m)==bool and  cv_m[4]<=ch_m[4]):
        c_m=cv_m.copy()
    if not 0 in tab:
        return -1,-1
    else:
        for i in range(tab.shape[0]):
            for j in range(tab.shape[1]):
                if tab[i,j]==0:
                    cv=search_constrain_v(constrain,i,j)
                    ch=search_constrain_h(constrain,i,j)
                    if ((not type(cv)==bool)) and cv[4]<c_m[4]:
                        m=(i,j)
                        c_m=cv.copy()
                    if ((not type(ch)==bool)  and ch[4]<c_m[4]):
                        m=(i,j)
                        c_m=ch.copy()
    return m
                    

In [15]:
tab3=tab.copy()
search(tab3,constrain,selection=select_cell_intelligent)

True

In [16]:
tab3

array([[-1, -1, -1, -1, -1],
       [-1, -1,  3,  1,  8],
       [-1,  3,  1,  2,  4],
       [-1,  1,  2, -1,  1],
       [-1, -1,  4,  1, -1]])